<a href="https://colab.research.google.com/github/echughes529/quantum_network_simulation/blob/main/streamlining_sequential_protocol_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install qutip

In [4]:
import numpy as np
import random
import matplotlib.pyplot as plt
from qutip import *

# Optical Fiber Properties
L_att = 22e3       # attenuation length of optical fiber
c = 2e8            # speed of light in fiber optic


# Set Up
zero_ket = basis(2,0)
one_ket = basis(2,1)
zero_rho = zero_ket * zero_ket.dag()
one_rho = basis(2,1) * basis(2,1).dag()
I = lambda: qeye(2)
H = (sigmaz() + sigmax()) * np.sqrt(1/2)
CNOT = Qobj([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]], dims=[[2,2],[2,2]])
phi_plus_ket = (tensor(one_ket, one_ket) + tensor(zero_ket, zero_ket)).unit()
phi_plus_rho = phi_plus_ket * phi_plus_ket.dag()


def initial_rho(num_qubits: int):
    assert isinstance(num_qubits, int) and num_qubits > 0, "num_qubits must be a positive integer."
    zero_rho = basis(2, 0) * basis(2, 0).dag()  # Density matrix for the |0⟩ state
    return tensor([zero_rho for _ in range(num_qubits)])


def pad_op(op: Qobj, left_padding: int, right_padding: int, num_qubits: int):
    assert isinstance(op, Qobj), "op must be a Qobj."
    assert isinstance(left_padding, int) and left_padding >= 0, "left_padding must be a non-negative integer."
    assert isinstance(right_padding, int) and right_padding >= 0, "right_padding must be a non-negative integer."

    if left_padding > 0:
        left_op = tensor([I()] * left_padding)
        op = tensor(left_op, op)

    if right_padding > 0:
        right_op = tensor([I()] * right_padding)
        op = tensor(op, right_op)

    return op

def phi_plus_generator(rho: Qobj, num_qubits_left: int, num_qubits_right: int, num_qubits: int, F_initial=0.93):
    H_op = pad_op(H, left_padding=num_qubits_left, right_padding=num_qubits_right + 1, num_qubits=num_qubits)
    CNOT_op = pad_op(CNOT, left_padding=num_qubits_left, right_padding=num_qubits_right, num_qubits=num_qubits)

    rho = H_op * rho * H_op.dag()
    rho = CNOT_op * rho * CNOT_op.dag()
    return rho


def dephasing_channel(rho: Qobj, t: float, T_dp: float, d: float, left_padding: int, right_padding: int, num_qubits: int):
    # convert time units to real life time units
    t = t * d / c
    dp_prob = (1 - np.exp(-t / T_dp)) / 2

    Z_op = np.sqrt(dp_prob) * pad_op(op=sigmaz(), left_padding=left_padding, right_padding=right_padding, num_qubits=num_qubits)
    I_op = np.sqrt(1 - dp_prob) * pad_op(op=I(), left_padding=left_padding, right_padding=right_padding, num_qubits=num_qubits)
    rho_t = I_op * rho * I_op.dag() + Z_op * rho * Z_op.dag()
    return rho_t


def entanglement_generation(rho: Qobj, q0: int, q1: int, num_qubits: int, T_dp: float, d: float, qubit_times: list):
    """
    NOTE: q1 must be greater than q0, q0 always even, q1 always dd
    Returns rho (either with entanglement generated or not) and True or False depending on whether entanglment has been generated
    time here is in time units of d/c, converted to real time units in dephasing function
    qubit times for record keeping purposesc
    """
    # proabilistic entanglement generation according to the channel efficiency
    n_channel = np.exp(-d/L_att)
    num_qubits_left = q0
    num_qubits_right = (num_qubits - 1) - q1

    # if unsuccessful, qubits to left decohere
    if random.uniform(0,1) > n_channel:
        print(f"Entanglement generation between qubits {q0} and {q1} failed.")
        if q0 > 0:
            print(f"Dephasing qubits 0 and {q0-1} due to failed attempt.")
            # dephasing on left of q0
            rho = dephasing_channel(rho=rho, t = 2, T_dp=T_dp, d=d, left_padding=num_qubits_left - 1, right_padding=num_qubits_right + 2, num_qubits=num_qubits)
            qubit_times[q0-1] += 2
            # dephasing on first qubit
            rho = dephasing_channel(rho=rho, t = 2, T_dp=T_dp, d=d, left_padding=0, right_padding=num_qubits - 1, num_qubits=num_qubits)
            qubit_times[0] += 2
        return rho, False

    else:
        print(f"Entanglement successfully generated between qubits {q0} and {q1}.")
        rho = phi_plus_generator(rho, num_qubits_left, num_qubits_right, num_qubits)

        # dephasing on q0
        rho = dephasing_channel(rho=rho, t = 2, T_dp=T_dp, d=d, left_padding=num_qubits_left, right_padding=num_qubits_right + 1, num_qubits=num_qubits)
        qubit_times[q0] += 2
        # dephasing on q1
        rho = dephasing_channel(rho=rho, t = 1, T_dp=T_dp, d=d, left_padding=num_qubits_left + 1, right_padding=num_qubits_right, num_qubits=num_qubits)
        qubit_times[q1] += 1

        # qubits to left must decohere (unless q0 = 0)
        if q0 > 0:
            print(f"Dephasing qubits 0 and {q0-1} due after succesful attempt.")
            # dephasing on left of q0
            rho = dephasing_channel(rho=rho, t = 2, T_dp=T_dp, d=d, left_padding=num_qubits_left - 1, right_padding=num_qubits_right + 2, num_qubits=num_qubits)
            qubit_times[q0-1] += 2
            # dephasing on first qubit
            rho = dephasing_channel(rho=rho, t = 2, T_dp=T_dp, d=d, left_padding=0, right_padding=num_qubits - 1, num_qubits=num_qubits)
            qubit_times[0] += 2
    return rho, True

def entanglement_swapping(rho: Qobj, q_minus_1: int, q0: int, num_qubits: int):
    print(f"Entanglement swapping between qubits {q_minus_1} and {q0}.")
    num_qubits_left = q_minus_1
    num_qubits_right = (num_qubits - 1) - q0

    bell_measure_op = pad_op(phi_plus_rho, left_padding = num_qubits_left, right_padding = num_qubits_right, num_qubits=num_qubits)
    rho = (bell_measure_op * rho * bell_measure_op.dag()).unit()
    return rho

# Parameters
T_dp = 0.000001
d = 5000
num_nodes = 6
num_iterations = 1
num_qubits = num_nodes * 2


In [ ]:
rho = initial_rho(num_qubits)
qubit_times = np.zeros(num_qubits)
q0, q1 = 0, 1

move_on = False
# Generate entanglement across the first pair
while move_on == False:
    rho, move_on = entanglement_generation(rho, q0, q1, num_qubits, T_dp, d, qubit_times)
    q0 += 2
    q1 += 2

# keep generating entanglement until last pair
while q0 <= num_qubits - 2:
    move_on = False
    while move_on == False:
        rho, move_on = entanglement_generation(rho, q0, q1, num_qubits, T_dp, d, qubit_times)
    rho = entanglement_swapping(rho, q0-1, q0, num_qubits)
    q0 += 2
    q1 += 2

# trace out final bell pair at the end nodes
final_state = rho.ptrace([0, num_qubits - 1])
fidelity = np.real(phi_plus_ket.dag() * final_state * phi_plus_ket)
print('final state:', final_state)
print('fidelity:', fidelity)


Entanglement successfully generated between qubits 0 and 1.


In [ ]:
phi_plus_ket = (tensor(one_ket, one_ket) + tensor(zero_ket, zero_ket)).unit()
phi_plus_rho = phi_plus_ket * phi_plus_ket.dag()

np.real(phi_plus_ket.dag() * phi_plus_rho * phi_plus_ket)
print(phi_plus_rho.full())
print(final_state.full())

In [ ]:
def run_communication_chain(num_iterations: int, T_dp: float, d: float, num_qubits: int):
    """Run the communication chain multiple times and calculate the average fidelity."""
    fidelities = []
    num_qubits = (num_nodes - 1) * 2

    for _ in range(num_iterations):
        rho = initial_rho(num_qubits)
        qubit_times = np.zeros(num_qubits)
        q0, q1 = 0, 1

        move_on = False
        # Generate entanglement across the first pair
        while move_on == False:
            rho, move_on = entanglement_generation(rho, q0, q1, num_qubits, T_dp, d, qubit_times)
            q0 += 2
            q1 += 2

        # keep generating entanglement until last pair
        while q0 <= num_qubits - 2:
            while move_on == False:
                rho, move_on = entanglement_generation(rho, q0, q1, num_qubits, T_dp, d, qubit_times)
            #rho = entanglement_swapping(rho, q0-1, q0, num_qubits)
            q0 += 2
            q1 += 2

        # trace out final bell pair at the end nodes
        final_state = rho.ptrace([0, num_qubits - 1])
        fidelity = np.real(phi_plus_ket.dag() * final_state * phi_plus_ket)
        fidelities.append(fidelity)

    avg_fidelity = np.mean(fidelities)
    return avg_fidelity



# Run the simulation
#avg_fidelity = run_communication_chain(num_iterations, T_dp, d, num_qubits)
#print(f"Average Fidelity: {avg_fidelity:.4f}")